In [ ]:
!pip install sklearn
!pip install segmentation_models
!pip install albumentations==0.4.5

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image

import os
import glob
import numpy as np

import skimage.transform as sk_transform
import skimage.filters as sk_filters
from skimage.measure import label, regionprops
from sklearn import model_selection

import segmentation_models as sm

In [ ]:
from src.DataLoader import DataLoader, make_augmentation
from src.Model import unet_model

## Load data

In [ ]:
base_dir = "/storage/supervisely/processed"
ims_dir = os.path.join(base_dir, "imgs")
labels_dir = os.path.join(base_dir, "labels")
img_files = sorted(glob.glob(ims_dir + "/*.*"))
mask_files = sorted(glob.glob(labels_dir + "/*.png"))
print (f"Masks count : {len(mask_files)};\nImgs count : {len(img_files)};")

In [ ]:
BATCH_SIZE = 24
dataset = list(zip(img_files, mask_files))
train_dataset, test_dataset = model_selection.train_test_split(dataset, test_size=0.2, random_state=0)
print(f"Train dataset size {len(train_dataset)}; Test dataset size {len(test_dataset)}")

In [ ]:
output_size = (256, 256)

training_augmentation = make_augmentation(output_size=output_size, is_validation=False)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, augmentation_fn=training_augmentation, output_size=output_size, shuffle=True)

validation_augmentation = make_augmentation(output_size=output_size, is_validation=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, augmentation_fn=validation_augmentation, output_size=output_size, shuffle=False)

In [ ]:
OUTPUT_CHANNELS = 1
model = unet_model(OUTPUT_CHANNELS)

In [ ]:
# model.load_weights("/storage/segmentation_training/models/best_model_full_unwrapped_from_scratch_lr00001_batch24.h5", by_name=True)

## Configure losses and compile a model

In [ ]:
LR = 0.0001
EPOCHS = 40
# define optomizer
optim = tf.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
# focal_loss = sm.losses.BinaryFocalLoss()
# total_loss = dice_loss + (1 * focal_loss)
total_loss = dice_loss

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/storage/training_result/models/best_model_full_unwrapped_from_scratch_lr00001_batch24.h5', save_weights_only=True, save_best_only=True, mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.TensorBoard(log_dir="/storage/training_result/models/logs_full_unwrapped_from_scratch_lr00001_batch24")
]

In [ ]:
history = model.fit(
    x = train_data_loader, 
    steps_per_epoch=len(train_data_loader), 
    epochs=EPOCHS, 
    callbacks=callbacks, 
    validation_data=test_data_loader, 
    validation_steps=len(test_data_loader),
    use_multiprocessing=False,
    workers=1
) 

# print("Done")

In [ ]:
# !mkdir /storage/final_model

In [ ]:
# model.save("/storage/final_model")

In [ ]:
model.save("/storage/final_model.h5")

In [ ]:
model.summary()

In [ ]:
#imgs, masks = test_data_loader[0]
imgs, masks = train_data_loader[0]

In [ ]:
batch_res = model.predict(imgs)

In [ ]:
plt.imshow(batch_res[0].squeeze())

In [ ]:
plt.hist(batch_res[0].squeeze().ravel(), bins=100)